In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('PR_PF_1900086.csv')

In [3]:
data

,PLATFORM_CODE,DATE (YYYY-MM-DDTHH:MI:SSZ),DATE_QC,LATITUDE (degree_north),LONGITUDE (degree_east),POSITION_QC,PRES (decibar),PRES_QC,PSAL (psu),PSAL_QC,TEMP (degree_Celsius),TEMP_QC,PRES_ADJUSTED (decibar),PRES_ADJUSTED_QC,TEMP_ADJUSTED (degree_Celsius),TEMP_ADJUSTED_QC,PSAL_ADJUSTED (psu),PSAL_ADJUSTED_QC
0,1900086,2001-07-24T11:07:31Z,1,-31.399,48.037,1,7.0,1,35.731,1,19.091,1,3.4,1,19.091,1,35.731,1
1,1900086,2001-07-24T11:07:31Z,1,-31.399,48.037,1,9.0,1,35.730,1,19.091,1,5.4,1,19.091,1,35.730,1
2,1900086,2001-07-24T11:07:31Z,1,-31.399,48.037,1,19.3,1,35.730,1,19.105,1,15.7,1,19.105,1,35.730,1
3,1900086,2001-07-24T11:07:31Z,1,-31.399,48.037,1,29.4,1,35.730,1,19.107,1,25.8,1,19.107,1,35.730,1
4,1900086,2001-07-24T11:07:31Z,1,-31.399,48.037,1,39.1,1,35.730,1,19.110,1,35.5,1,19.110,1,35.730,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,1900086,2002-05-01T12:05:29Z,1,-32.390,46.177,1,1699.3,1,34.579,1,2.976,1,1695.4,1,2.976,1,34.579,1
1416,1900086,2002-05-01T12:05:29Z,1,-32.390,46.177,1,1799.0,1,34.611,1,2.838,1,1795.1,1,2.838,1,34.611,1
1417,1900086,2002-05-01T12:05:29Z,1,-32.390,46.177,1,1898.7,1,34.642,1,2.743,1,1894.8,1,2.743,1,34.642,1
1418,1900086,2002-05-01T12:05:29Z,1,-32.390,46.177,1,1999.2,1,34.684,1,2.612,1,1995.3,1,2.612,1,34.684,1


In [5]:
index = data['DATE (YYYY-MM-DDTHH:MI:SSZ)'].unique()

In [7]:
float_number = data.iloc[0,0]

In [29]:
import os
import glob

def get_all_csv_files(directory):
    # Join the directory path and the pattern for .csv files
    csv_files = glob.glob(os.path.join(directory, "*.csv"))
    
    return csv_files



print("CSV files found:")
for file in csv_files:
    print(file)


CSV files found:
../data\PR_PF_1900033.csv
../data\PR_PF_1900083.csv
../data\PR_PF_1900084.csv
../data\PR_PF_1900085.csv
../data\PR_PF_1900086.csv
../data\PR_PF_1900090.csv
../data\PR_PF_1900103.csv
../data\PR_PF_1900104.csv
../data\PR_PF_1900138.csv
../data\PR_PF_1900139.csv
../data\PR_PF_1900140.csv
../data\PR_PF_1900141.csv
../data\PR_PF_1900142.csv
../data\PR_PF_1900173.csv
../data\PR_PF_1900175.csv
../data\PR_PF_1900178.csv
../data\PR_PF_1900277.csv
../data\PR_PF_1900278.csv
../data\PR_PF_1900279.csv
../data\PR_PF_1900280.csv
../data\PR_PF_1900286.csv
../data\PR_PF_1900289.csv
../data\PR_PF_1900290.csv
../data\PR_PF_1900292.csv
../data\PR_PF_1900293.csv
../data\PR_PF_1900305.csv
../data\PR_PF_1900306.csv
../data\PR_PF_1900307.csv
../data\PR_PF_1900308.csv
../data\PR_PF_1900309.csv
../data\PR_PF_1900310.csv
../data\PR_PF_1900311.csv
../data\PR_PF_1900343.csv
../data\PR_PF_1900344.csv
../data\PR_PF_1900345.csv
../data\PR_PF_1900346.csv
../data\PR_PF_1900364.csv
../data\PR_PF_1900368

In [69]:

# Example usage:
directory_path = '../data/'
csv_files = get_all_csv_files(directory_path)
for file in csv_files:
    print(file)
    data = pd.read_csv(file)
    data.dropna(inplace=True)
    data = data.reset_index()
    if not data.empty:
        index = data['DATE (YYYY-MM-DDTHH:MI:SSZ)'].unique()
        float_number = data['PLATFORM_CODE'].iloc[0]
        
        agg_data = {
            'float_number': []
            ,'date':[]
            ,'latitude':[]
            ,'longitude':[]
            ,'q0_psal':[]
            ,'q1_psal':[]
            ,'q2_psal':[]
            ,'q3_psal':[]
            ,'q4_psal':[]
            ,'q0_temp':[]
            ,'q1_temp':[]
            ,'q2_temp':[]
            ,'q3_temp':[]
            ,'q4_temp':[]
        }

        if 'PSAL (psu)' in data.columns:
            for i in index:
                subset = data[data['DATE (YYYY-MM-DDTHH:MI:SSZ)']==i]
                agg_data['float_number'].append(float_number)
                agg_data['date'].append(i)
                agg_data['latitude'].append(subset['LATITUDE (degree_north)'].iloc[0])
                agg_data['longitude'].append(subset['LONGITUDE (degree_east)'].iloc[0])
                q0 = min(subset['PRES (decibar)'])
                q1 = min(subset['PRES (decibar)'], key=lambda x:abs(x-500))
                q2 = min(subset['PRES (decibar)'], key=lambda x:abs(x-1000))
                q3 = min(subset['PRES (decibar)'], key=lambda x:abs(x-1500))
                q4 = max(subset['PRES (decibar)'])
                agg_data['q0_psal'].append(subset[subset['PRES (decibar)']==q0]['PSAL (psu)'].iloc[0])
                agg_data['q1_psal'].append(subset[subset['PRES (decibar)']==q1]['PSAL (psu)'].iloc[0])
                agg_data['q2_psal'].append(subset[subset['PRES (decibar)']==q2]['PSAL (psu)'].iloc[0])
                agg_data['q3_psal'].append(subset[subset['PRES (decibar)']==q3]['PSAL (psu)'].iloc[0])
                agg_data['q4_psal'].append(subset[subset['PRES (decibar)']==q4]['PSAL (psu)'].iloc[0])
                agg_data['q0_temp'].append(subset[subset['PRES (decibar)']==q0]['TEMP (degree_Celsius)'].iloc[0])
                agg_data['q1_temp'].append(subset[subset['PRES (decibar)']==q1]['TEMP (degree_Celsius)'].iloc[0])
                agg_data['q2_temp'].append(subset[subset['PRES (decibar)']==q2]['TEMP (degree_Celsius)'].iloc[0])
                agg_data['q3_temp'].append(subset[subset['PRES (decibar)']==q3]['TEMP (degree_Celsius)'].iloc[0])
                agg_data['q4_temp'].append(subset[subset['PRES (decibar)']==q4]['TEMP (degree_Celsius)'].iloc[0])
            agg_data = pd.DataFrame(agg_data)
            agg_data['date'] = pd.to_datetime(agg_data['date'])
            agg_data['date'] = agg_data['date'].dt.strftime('%Y-%m-%d') 
            all_data = pd.read_csv('all_ocean_data.csv')
            all_data = pd.concat([agg_data, all_data])
            all_data.to_csv('all_ocean_data.csv', index=False)




../data\PR_PF_1900033.csv
../data\PR_PF_1900083.csv
../data\PR_PF_1900084.csv
../data\PR_PF_1900085.csv
../data\PR_PF_1900086.csv
../data\PR_PF_1900090.csv
../data\PR_PF_1900103.csv
../data\PR_PF_1900104.csv
../data\PR_PF_1900138.csv
../data\PR_PF_1900139.csv
../data\PR_PF_1900140.csv
../data\PR_PF_1900141.csv
../data\PR_PF_1900142.csv
../data\PR_PF_1900173.csv
../data\PR_PF_1900175.csv
../data\PR_PF_1900178.csv
../data\PR_PF_1900277.csv
../data\PR_PF_1900278.csv
../data\PR_PF_1900279.csv
../data\PR_PF_1900280.csv
../data\PR_PF_1900286.csv
../data\PR_PF_1900289.csv
../data\PR_PF_1900290.csv
../data\PR_PF_1900292.csv
../data\PR_PF_1900293.csv
../data\PR_PF_1900305.csv
../data\PR_PF_1900306.csv
../data\PR_PF_1900307.csv
../data\PR_PF_1900308.csv
../data\PR_PF_1900309.csv
../data\PR_PF_1900310.csv
../data\PR_PF_1900311.csv
../data\PR_PF_1900343.csv
../data\PR_PF_1900344.csv
../data\PR_PF_1900345.csv
../data\PR_PF_1900346.csv
../data\PR_PF_1900364.csv
../data\PR_PF_1900368.csv
../data\PR_P

KeyboardInterrupt: 

In [68]:
print(file)
data = pd.read_csv('PR_PF_1900086.csv')
index = data['DATE (YYYY-MM-DDTHH:MI:SSZ)'].unique()


agg_data = {
    'float_number': []
    ,'date':[]
    ,'latitude':[]
    ,'longitude':[]
    ,'q0_psal':[]
    ,'q1_psal':[]
    ,'q2_psal':[]
    ,'q3_psal':[]
    ,'q4_psal':[]
    ,'q0_temp':[]
    ,'q1_temp':[]
    ,'q2_temp':[]
    ,'q3_temp':[]
    ,'q4_temp':[]
}

if 'PSAL (psu)' in data.columns:
    for i in index:
        subset = data[data['DATE (YYYY-MM-DDTHH:MI:SSZ)']==i]
        agg_data['float_number'].append(float_number)
        agg_data['date'].append(i)
        agg_data['latitude'].append(subset['LATITUDE (degree_north)'].iloc[0])
        agg_data['longitude'].append(subset['LONGITUDE (degree_east)'].iloc[0])
        q0 = min(subset['PRES (decibar)'])
        q1 = min(subset['PRES (decibar)'], key=lambda x:abs(x-500))
        q2 = min(subset['PRES (decibar)'], key=lambda x:abs(x-1000))
        q3 = min(subset['PRES (decibar)'], key=lambda x:abs(x-1500))
        q4 = max(subset['PRES (decibar)'])
        agg_data['q0_psal'].append(subset[subset['PRES (decibar)']==q0]['PSAL (psu)'].iloc[0])
        agg_data['q1_psal'].append(subset[subset['PRES (decibar)']==q1]['PSAL (psu)'].iloc[0])
        agg_data['q2_psal'].append(subset[subset['PRES (decibar)']==q2]['PSAL (psu)'].iloc[0])
        agg_data['q3_psal'].append(subset[subset['PRES (decibar)']==q3]['PSAL (psu)'].iloc[0])
        agg_data['q4_psal'].append(subset[subset['PRES (decibar)']==q4]['PSAL (psu)'].iloc[0])
        agg_data['q0_temp'].append(subset[subset['PRES (decibar)']==q0]['TEMP (degree_Celsius)'].iloc[0])
        agg_data['q1_temp'].append(subset[subset['PRES (decibar)']==q1]['TEMP (degree_Celsius)'].iloc[0])
        agg_data['q2_temp'].append(subset[subset['PRES (decibar)']==q2]['TEMP (degree_Celsius)'].iloc[0])
        agg_data['q3_temp'].append(subset[subset['PRES (decibar)']==q3]['TEMP (degree_Celsius)'].iloc[0])
        agg_data['q4_temp'].append(subset[subset['PRES (decibar)']==q4]['TEMP (degree_Celsius)'].iloc[0])
    agg_data = pd.DataFrame(agg_data)
    agg_data['date'] = pd.to_datetime(agg_data['date'])
    agg_data['date'] = agg_data['date'].dt.strftime('%Y-%m-%d') 

    agg_data.to_csv('all_ocean_data.csv', index=False)

../data\PR_PF_1900508.csv


In [57]:
print(file)
data = pd.read_csv('PR_PF_1900086.csv')
index = data['DATE (YYYY-MM-DDTHH:MI:SSZ)'].unique()


agg_data = {
    'float_number': []
    ,'date':[]
    ,'latitude':[]
    ,'longitude':[]
    ,'q0_psal':[]
    ,'q1_psal':[]
    ,'q2_psal':[]
    ,'q3_psal':[]
    ,'q4_psal':[]
    ,'q0_temp':[]
    ,'q1_temp':[]
    ,'q2_temp':[]
    ,'q3_temp':[]
    ,'q4_temp':[]
}

if 'PSAL (psu)' in data.columns:
    for i in index:
        subset = data[data['DATE (YYYY-MM-DDTHH:MI:SSZ)']==i]
        agg_data['float_number'].append(float_number)
        agg_data['date'].append(i)
        agg_data['latitude'].append(subset['LATITUDE (degree_north)'].iloc[0])
        agg_data['longitude'].append(subset['LONGITUDE (degree_east)'].iloc[0])
        q0 = min(subset['PRES (decibar)'])
        q1 = min(subset['PRES (decibar)'], key=lambda x:abs(x-500))
        q2 = min(subset['PRES (decibar)'], key=lambda x:abs(x-1000))
        q3 = min(subset['PRES (decibar)'], key=lambda x:abs(x-1500))
        q4 = max(subset['PRES (decibar)'])
        agg_data['q0_psal'].append(subset[subset['PRES (decibar)']==q0]['PSAL (psu)'].iloc[0])
        agg_data['q1_psal'].append(subset[subset['PRES (decibar)']==q1]['PSAL (psu)'].iloc[0])
        agg_data['q2_psal'].append(subset[subset['PRES (decibar)']==q2]['PSAL (psu)'].iloc[0])
        agg_data['q3_psal'].append(subset[subset['PRES (decibar)']==q3]['PSAL (psu)'].iloc[0])
        agg_data['q4_psal'].append(subset[subset['PRES (decibar)']==q4]['PSAL (psu)'].iloc[0])
        agg_data['q0_temp'].append(subset[subset['PRES (decibar)']==q0]['TEMP (degree_Celsius)'].iloc[0])
        agg_data['q1_temp'].append(subset[subset['PRES (decibar)']==q1]['TEMP (degree_Celsius)'].iloc[0])
        agg_data['q2_temp'].append(subset[subset['PRES (decibar)']==q2]['TEMP (degree_Celsius)'].iloc[0])
        agg_data['q3_temp'].append(subset[subset['PRES (decibar)']==q3]['TEMP (degree_Celsius)'].iloc[0])
        agg_data['q4_temp'].append(subset[subset['PRES (decibar)']==q4]['TEMP (degree_Celsius)'].iloc[0])
    agg_data = pd.DataFrame(agg_data)
    agg_data['date'] = pd.to_datetime(agg_data['date'])
    agg_data['date'] = agg_data['date'].dt.strftime('%Y-%m-%d') 
    all_data = pd.read_csv('all_ocean_data.csv')
    # print(agg_data.head(), all_data.head())
    all_data = pd.concat([agg_data, all_data])
    all_data.to_csv('all_ocean_data.csv', index=False)

../data\PR_PF_1900508.csv
